###  Data Integration Project

Transaction and Article csv files cleaned, validated and combined to create target table using PySpark.

#### Import Libraries & Read Csv Files

In [ ]:
import pyspark
from pyspark.sql.functions import col, isnan, count, when, regexp_replace, to_date, monotonically_increasing_id
from pyspark.sql.types import IntegerType, FloatType

In [ ]:
def read_csv_dbfs(file_name, infer_schema, first_row_is_header, delimiter):
    """
    Reads csv file from DBFS's /FileStore/tables/ location into PySpark DataFrame.
    return: df 
    """
    file_location = "/FileStore/tables/" + file_name
    file_type = "csv"
    # The applied options are for CSV files. For other file types, these will be ignored.
    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .load(file_location)
    
    return df

In [ ]:
def describe_df(df):
    """
    Shows a dataframe's schema, shape and distinct-duplicate row counts
    """
    # Schema
    df.printSchema()
    # Shape
    print(f"Row Count: {df.count()}, Column Count: {len(df.columns)}")
    # Distinct & Duplicate row count
    distinct_row_count = df.distinct().count()
    duplicate_row_count = df.count() - df.distinct().count()
    print(f"Distinct Row Count: {distinct_row_count}, Duplicate Row Count: {duplicate_row_count}")

##### Read and Describe TRANSACTION data

In [ ]:
# Read TRANSACTION.csv into dataframe
transaction_df = read_csv_dbfs(file_name="TRANSACTION.csv", infer_schema="true", first_row_is_header="true", delimiter=",")

In [ ]:
# Describe Transaction DF
describe_df(transaction_df)

root
-- TRANSACTION_ID/SOL/BONKEY: decimal(22,0) (nullable = true)
-- TRANSACTION_TIME0RPA_ETS2: integer (nullable = true)
-- SALESORG0SALESORG: integer (nullable = true)
-- TRANSACTION_DATE0CALDAY: integer (nullable = true)
-- DISTR_CHAN0DISTR_CHAN: integer (nullable = true)
-- ARTICLE_ID0MATERIAL: long (nullable = true)
-- LOCATION_ID0PLANT: integer (nullable = true)
-- TRANSACTION_TYPE0RPA_TTC: integer (nullable = true)
-- ARTICLE_COUNT0RPA_RLQ: string (nullable = true)
-- SALES_PRICE_AT_CASH_DESK0RPA_SAT: string (nullable = true)
-- SALES_PRICE_PLANNED/SOL/LOC0086C: string (nullable = true)
-- VAT0RPA_TAM: string (nullable = true)

Row Count: 14779, Column Count: 12
Distinct Row Count: 14779, Duplicate Row Count: 0

##### Read and Describe ARTICLE data

In [ ]:
# Read TRANSACTION.csv into dataframe
article_df = read_csv_dbfs(file_name="ARTICLE.csv", infer_schema="true", first_row_is_header="true", delimiter=",")

In [ ]:
# Describe ARTICLE DF
describe_df(article_df)

root
-- ARTICLE_ID0MATERIAL: string (nullable = true)
-- ARTICLE_COLOR_ID0RT_COLOR: string (nullable = true)
-- ARTICLE_GROUP_ID0RT_CONFMAT: string (nullable = true)
-- EAN0EANUPC: string (nullable = true)
-- DESCRIPTION0TXTMD: string (nullable = true)
-- BRAND_NAME/SOL/MDPROD1: integer (nullable = true)
-- PICTURE_PATH0EXT_URL: string (nullable = true)
-- INITIAL_SEASON_NAME/SOL/FMSSEASO: integer (nullable = true)
-- CURRENT_SEASON_NAME: integer (nullable = true)
-- MATL_TYPE0MATL_TYPE: string (nullable = true)

Row Count: 223893, Column Count: 10
Distinct Row Count: 223893, Duplicate Row Count: 0

### Cleaning Transaction Data

- Unwanted float values and int values like "31.99-"" and "1.000-" detected at "SALES_PRICE_AT_CASH_DESK0RPA_SAT","SALES_PRICE_PLANNED/SOL/LOC0086C","VAT0RPA_TAM" and "ARTICLE_COUNT0RPA_RLQ" columns. This values transformed and casted to desired format.

In [ ]:
transaction_float_columns=["SALES_PRICE_AT_CASH_DESK0RPA_SAT","SALES_PRICE_PLANNED/SOL/LOC0086C","VAT0RPA_TAM"]
transaction_int_columns = ["ARTICLE_COUNT0RPA_RLQ"]

In [ ]:
# All numeric values can be casted to int so cast("int") used,  
def transform_numeric_columns(df, column_list):
    """
    Dirty rows like  "31.99-" validated to a float like string.
    return: df
    """
    for column in transaction_numeric_columns:
        df = df.withColumn(column, when(col(column).cast("int").isNull(),regexp_replace(column, '-', '')).otherwise(col(column)))
    
    return df
    

In [ ]:
def cast_numeric_columns(df, column_list, data_type):
    """
    Cast Columns to desired data types if all rows are validated.
    return: df
    """
    for column in column_list:
        unmatched_row_count = df.filter(col(column).cast(data_type).isNull()).count()
        print(f"Column Name: {column}, Not{data_type} row Count: {unmatched_row_count}")
        if unmatched_row_count == 0:
            print(f"All values can be casted to {data_type} data type.")
            df = df.withColumn(column,col(column).cast(data_type))
        else:
            print("Ups ! Some values still include dirty characters like: ")
            print(df.filter(col(column).cast(data_type).isNull()).show())
    return df

In [ ]:
def duplicate_controller(df):
    """
    Drops duplicate rows if exists
    return: df
    """
    distinct_row_count = df.distinct().count()
    duplicate_row_count = df.count() - distinct_row_count
    if duplicate_row_count > 0:
        df = df.dropDuplicates(targettable_df.columns)
        print("Droped duplicate rows")
    else :
        print("No duplicates")
    return df

In [ ]:
# Validate Transaction Data and fix rows
transaction_numeric_columns = transaction_int_columns + transaction_float_columns
transaction_df = transform_numeric_columns(transaction_df, transaction_numeric_columns)
# Cast columns to desired data types
transaction_df = cast_numeric_columns(transaction_df, transaction_float_columns, "float")
transaction_df = cast_numeric_columns(transaction_df, transaction_int_columns, "int")

Column Name: SALES_PRICE_AT_CASH_DESK0RPA_SAT, Notfloat row Count: 0
All values can be casted to float data type.
Column Name: SALES_PRICE_PLANNED/SOL/LOC0086C, Notfloat row Count: 0
All values can be casted to float data type.
Column Name: VAT0RPA_TAM, Notfloat row Count: 0
All values can be casted to float data type.
Column Name: ARTICLE_COUNT0RPA_RLQ, Notint row Count: 0
All values can be casted to int data type.

In [ ]:
#check dtypes
transaction_df.dtypes

Out[13]: [('TRANSACTION_ID/SOL/BONKEY', 'decimal(22,0)'),
 ('TRANSACTION_TIME0RPA_ETS2', 'int'),
 ('SALESORG0SALESORG', 'int'),
 ('TRANSACTION_DATE0CALDAY', 'int'),
 ('DISTR_CHAN0DISTR_CHAN', 'int'),
 ('ARTICLE_ID0MATERIAL', 'bigint'),
 ('LOCATION_ID0PLANT', 'int'),
 ('TRANSACTION_TYPE0RPA_TTC', 'int'),
 ('ARTICLE_COUNT0RPA_RLQ', 'int'),
 ('SALES_PRICE_AT_CASH_DESK0RPA_SAT', 'float'),
 ('SALES_PRICE_PLANNED/SOL/LOC0086C', 'float'),
 ('VAT0RPA_TAM', 'float')]

### Cleaning Article Dataset

In [ ]:
# Barcode Ean value should be bigint or long but null values breaks the type so null values replaced and column casted to long
article_long_columns = ["EAN0EANUPC"]

for column in article_long_columns:
    article_df = article_df.withColumn(column, when(col(column).cast("bigint").isNull(),regexp_replace(column, 'null', '0')).otherwise(col(column)))
    
article_df = cast_numeric_columns(article_df, article_long_columns, "long")

Column Name: EAN0EANUPC, Notlong row Count: 0
All values can be casted to long data type.

In [ ]:
article_df.dtypes

Out[15]: [('ARTICLE_ID0MATERIAL', 'string'),
 ('ARTICLE_COLOR_ID0RT_COLOR', 'string'),
 ('ARTICLE_GROUP_ID0RT_CONFMAT', 'string'),
 ('EAN0EANUPC', 'bigint'),
 ('DESCRIPTION0TXTMD', 'string'),
 ('BRAND_NAME/SOL/MDPROD1', 'int'),
 ('PICTURE_PATH0EXT_URL', 'string'),
 ('INITIAL_SEASON_NAME/SOL/FMSSEASO', 'int'),
 ('CURRENT_SEASON_NAME', 'int'),
 ('MATL_TYPE0MATL_TYPE', 'string')]

###Join Transaction and Article Dataframes by Article_Id

In [ ]:
#Column names simplified for better understanding
transaction_df=transaction_df.withColumnRenamed("ARTICLE_ID0MATERIAL","FKARTICLE_ID") \
.withColumnRenamed("TRANSACTION_ID/SOL/BONKEY","TRANSACTION_ID")

In [ ]:
# Joined two dataframes on Article_id column. As Article_id defined as Foreign Key, records that have matching values in both dataframes joined using inner join method. 
targettable_df = transaction_df.join(article_df, transaction_df.FKARTICLE_ID == article_df.ARTICLE_ID0MATERIAL, how= "inner").drop("ARTICLE_ID0MATERIAL")

In [ ]:
# Describe Transaction DF
describe_df(targettable_df)

root
-- TRANSACTION_ID: decimal(22,0) (nullable = true)
-- TRANSACTION_TIME0RPA_ETS2: integer (nullable = true)
-- SALESORG0SALESORG: integer (nullable = true)
-- TRANSACTION_DATE0CALDAY: integer (nullable = true)
-- DISTR_CHAN0DISTR_CHAN: integer (nullable = true)
-- FKARTICLE_ID: long (nullable = true)
-- LOCATION_ID0PLANT: integer (nullable = true)
-- TRANSACTION_TYPE0RPA_TTC: integer (nullable = true)
-- ARTICLE_COUNT0RPA_RLQ: integer (nullable = true)
-- SALES_PRICE_AT_CASH_DESK0RPA_SAT: float (nullable = true)
-- SALES_PRICE_PLANNED/SOL/LOC0086C: float (nullable = true)
-- VAT0RPA_TAM: float (nullable = true)
-- ARTICLE_COLOR_ID0RT_COLOR: string (nullable = true)
-- ARTICLE_GROUP_ID0RT_CONFMAT: string (nullable = true)
-- EAN0EANUPC: long (nullable = true)
-- DESCRIPTION0TXTMD: string (nullable = true)
-- BRAND_NAME/SOL/MDPROD1: integer (nullable = true)
-- PICTURE_PATH0EXT_URL: string (nullable = true)
-- INITIAL_SEASON_NAME/SOL/FMSSEASO: integer (nullable = true)
-- CURRENT_SEASON_NAME: integer (nullable = true)
-- MATL_TYPE0MATL_TYPE: string (nullable = true)

Row Count: 7184, Column Count: 21
Distinct Row Count: 7184, Duplicate Row Count: 0

In [ ]:
# Cast Integer type TRANSACTION_DATE column to date type
targettable_df = targettable_df.withColumn("TRANSACTION_DATE0CALDAY",col("TRANSACTION_DATE0CALDAY").cast("string")).withColumn("TRANSACTION_DATE",to_date(col("TRANSACTION_DATE0CALDAY"),"yyyyMMdd").cast("date")).drop("TRANSACTION_DATE0CALDAY")

In [ ]:
# Sort by date, than transaction time -> date aggregation ?
targettable_df = targettable_df.sort(col("TRANSACTION_DATE").asc(),col("TRANSACTION_TIME0RPA_ETS2").asc())

In [ ]:
# Check if duplicate rows exists and drops them if exists
targettable_df = duplicate_controller(targettable_df)

No duplicates

In [ ]:
# As some TransactionId values occured multiple times with different Articles a new primary index key added to dataframe.  
targettable_df = targettable_df.withColumn("INDEX",monotonically_increasing_id())

In [ ]:
# Final dataframe overview
display(targettable_df)

TRANSACTION_ID,TRANSACTION_TIME0RPA_ETS2,SALESORG0SALESORG,DISTR_CHAN0DISTR_CHAN,FKARTICLE_ID,LOCATION_ID0PLANT,TRANSACTION_TYPE0RPA_TTC,ARTICLE_COUNT0RPA_RLQ,SALES_PRICE_AT_CASH_DESK0RPA_SAT,SALES_PRICE_PLANNED/SOL/LOC0086C,VAT0RPA_TAM,ARTICLE_COLOR_ID0RT_COLOR,ARTICLE_GROUP_ID0RT_CONFMAT,EAN0EANUPC,DESCRIPTION0TXTMD,BRAND_NAME/SOL/MDPROD1,PICTURE_PATH0EXT_URL,INITIAL_SEASON_NAME/SOL/FMSSEASO,CURRENT_SEASON_NAME,MATL_TYPE0MATL_TYPE,TRANSACTION_DATE,INDEX
2021040660480000000018,73108,1099,13,2061332020,6048,1005,1,37.49,49.99,0.0,5760,000000000002061332,4065206011492,5223833e5b8fb5a4ce7b2b78b0a9cfe18c77e519,10,fc4b0394df1dfdff72c996ae898b99ada8b95d3e,202103,202103,ZMO3,2021-04-06,0
2021040660480000000015,73112,1099,13,2063756004,6048,1005,1,31.99,39.99,0.0,5959,000000000002063756,4065206022993,774f42a810ca602872e9ca74aa08255f0afe1ec1,10,463e1051034ff34b6062dc31c5e7d87413810b3d,202103,202103,ZMO3,2021-04-06,1
2021040660480000000013,73126,1099,13,2040421030,6048,1005,1,39.79,49.99,0.0,9400,000000000002040421,4063614074085,2a57c4e5da1a3187c6232f7ecb72b37ca45b9c83,10,ca9ef352a5419fbccd4bfbdcbfcfc442fe05d820,202009,202009,ZMO3,2021-04-06,2
2021040660480000000012,73135,1099,13,2064691023,6048,1005,1,43.92,59.99,0.0,59Z6,000000000002064691,4063615258798,f58bf0e43af3ca930db64f053cb292b1b1414418,10,c90323514196b56f6036cdcf87a7a89d9ad14af0,202101,202101,ZMO3,2021-04-06,3
2021040660480000000017,73150,1099,13,2064590005,6048,1005,1,22.49,29.99,0.0,0100,000000000002064590,4065206040188,eae9c711aad183ceb4b69995c0e4322c85b24a11,10,70437f3cc17a55194e7841426a86b91424fd5ca6,202103,202103,ZMO3,2021-04-06,4
2021040660480000000023,75816,1099,13,2061332020,6048,1005,1,39.99,49.99,0.0,5760,000000000002061332,4065206011492,5223833e5b8fb5a4ce7b2b78b0a9cfe18c77e519,10,fc4b0394df1dfdff72c996ae898b99ada8b95d3e,202103,202103,ZMO3,2021-04-06,5
2021040660480000000030,75826,1099,13,2061332016,6048,1005,1,49.99,49.99,0.0,3781,000000000002061332,4065206011454,5223833e5b8fb5a4ce7b2b78b0a9cfe18c77e519,10,416d33a140805abe83b3dbb9548219e80f9c3bef,202103,202103,ZMO3,2021-04-06,6
2021040660480000000036,75836,1099,13,2064625023,6048,1005,1,20.79,25.99,0.0,7940,000000000002064625,4063615256541,860ad9b1fa23204364bbbadcdde53b3b761b1aeb,10,53f6f2652d423a7ff0dfd9ac7896583d323400de,202102,202102,ZMO3,2021-04-06,7
2021040660480000000021,75850,1099,13,2058758006,6048,1005,1,57.06,69.99,0.0,53Z5,000000000002058758,4063615262764,579b5a82f37da61f8ec033e177c5316fc2b1e4ad,10,897501077fd316b2d8498ca1932e7ebb673434b0,202102,202102,ZMO3,2021-04-06,8
2021040660480000000028,75856,1099,13,2063722037,6048,1005,1,59.99,79.99,0.0,7940,000000000002063722,4063615053577,858d6b6f4947361fd1f75453a346b6924604f0df,10,3de73f0734ad4cdad439b27a14903d59cc8b9d9c,202101,202101,ZMO3,2021-04-06,9


In [ ]:
# Write final dataframe to target table
targettable_df.write.saveAsTable("TransactionArticle")